This notebook is just a PoC to check if it's possible to clasify vulnerable code.


## Datasets:

There  are  two  widely  used  sources  ofvulnerability  data  maintained  by  the  NIST:  the  NVD  which  contains  vulnerabilities  in  production  software,  andthe  SARD  project which  contains  production,  synthetic,and  academic  security  flaws  or  vulnerabilities.  In  the  NVD,each  vulnerability  has  a  unique  Common  Vulnerabilities  andExposures  IDentifier  (CVE  ID)  and  a  Common  WeaknessEnumeration  IDentifier  (CWE  ID)  that  indicates  the  type  ofthe  vulnerability  in  question.  We  collect  the  programs  thatcontain  one  or  multiple  vulnerabilities.  In  the  SARD,  eachprogram (i.e., test case) corresponds to one or multiple CWEIDs because a program can have multiple types of vulnerabil-ities. Therefore, programs with one or multiple CWE IDs arecollected.

https://nvd.nist.gov/vuln/search

https://samate.nist.gov/SARD/search.php 

Once the dataset has been downloaded use the following commands to save all files in one folder:

```bash
$ mkdir all/
$ find . -type f \( -iname "*.php" \) -exec mv {} all/ \;
$ ls all | wc -l
```

# Remove comentaries .java
os.system(" find . \( -type d -name .git -prune \) -o -type f -print0 | xargs -0 sed -i 's/[\/*]//g' ")

### Common vulnerabilities

At the moment, this tool can discover the following vulnerabilities:

- PHP:
    - CWE-810: OWASP Top Ten 2010 Category A1 - Injection



## Algorithms

Machine  learning  algorithms  cannot  learn  from  raw  text;so  several  steps  of  processing  the  dataset  are  needed  beforetraining  the  algorithm.  First,  the  code  snippets  need  to  beconverted  into  numerical  feature  vectors.  We  used  bag-of-word  model,  a  method  to  represent  each  unique  word  as  afeature. Second, we selected features extracted a subset fromthe dataset and used them for training the machine algorithm.


## Papers y articulos basados:

https://arxiv.org/pdf/1801.01681.pdf

https://arxiv.org/pdf/1803.04497.pdf

https://realpython.com/python-keras-text-classification/

https://arxiv.org/pdf/1809.07945.pdf

https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f

## Code to create dataset with PHP files 

- CWE-810: OWASP Top Ten 2010 Category A1 - Injection
- 4913 Files

In [17]:
import os
import json
import re

dirlist_sqli_php = os.listdir("datasets/sqli_php/")

datasets = []
dataset = {"id": "", "vulnerable":"", "code": ""}
dataset_id = 0
for file in dirlist_sqli_php:
    with open("datasets/sqli_php/"+file, "r") as f:
        code = f.read()
        
        # Remove comments
        code = re.sub('\/[^)]*\*\/', '', code)
        
        # Get vars (without $_POST) and remove duplicates
        vars_list = re.findall('[$](?!_)\S\w+', code)
        vars_list = list(set(vars_list))
        
        # Create dict with old and new vars
        count = 0
        vars_dict = {}
        for var in vars_list:
            old_var = var
            new_var = "$var"+str(count)
            vars_dict[old_var] = new_var
            count = count + 1
        #print(vars_dict)
        
        # Replace vars
        for old_var, new_var in vars_dict.items():
            code = code.replace(old_var, new_var)     
        #print(code)
        
        # Print one code
        if dataset_id == 1:
            print(code)
        
        # Remove linebreaks, tabs and double quotes to add into json
        code = code.replace("\n", "\\n").replace("\"", "\\\"").replace("\t", "\\t")
        
        dataset['code'] = code
        dataset['vulnerable'] = "True"
        
    dataset['id'] = dataset_id
    dataset_id = dataset_id + 1
    dataset_json = json.dumps(dataset)
    datasets.append(dataset_json)
print(datasets[:10])        


<?php



$var2 = $_GET['UserData'];

$var3 = filter_var($var2, FILTER_SANITIZE_EMAIL);
if (filter_var($var3, FILTER_VALIDATE_EMAIL))
  $var2 = $var3 ;
else
  $var2 = "" ;

$var0 = "cat ' $var2 '";

//flaw
$var1 = system($var0);


?>
['{"id": 0, "vulnerable": "True", "code": "<?php\\\\n\\\\n\\\\n\\\\n$var2 = $_GET[\'UserData\'];\\\\n\\\\n$var3 = filter_var($var2, FILTER_SANITIZE_EMAIL);\\\\nif (filter_var($var3, FILTER_VALIDATE_EMAIL))\\\\n  $var2 = $var3 ;\\\\nelse\\\\n  $var2 = \\\\\\"\\\\\\" ;\\\\n\\\\n$var0 = \\\\\\"cat \'\\\\\\". $var2 . \\\\\\"\'\\\\\\";\\\\n\\\\n//flaw\\\\n$var1 = system($var0);\\\\n\\\\n\\\\n?>"}', '{"id": 1, "vulnerable": "True", "code": "<?php\\\\n\\\\n\\\\n\\\\n$var2 = $_GET[\'UserData\'];\\\\n\\\\n$var3 = filter_var($var2, FILTER_SANITIZE_EMAIL);\\\\nif (filter_var($var3, FILTER_VALIDATE_EMAIL))\\\\n  $var2 = $var3 ;\\\\nelse\\\\n  $var2 = \\\\\\"\\\\\\" ;\\\\n\\\\n$var0 = \\\\\\"cat \' $var2 \'\\\\\\";\\\\n\\\\n//flaw\\\\n$var1 = system($var0);\\\\n\\\\n\\\

In [71]:
import pandas as pd
import os

dirlist_sqli_java = os.listdir("datasets/sqli_java/")
print("[*] " + str(len(dirlist)) + " files are importing...")


df_list = []
for file in dirlist_sqli_java:
    df = pd.read_csv("datasets/sqli_java/"+file, names=['code'])
    df['vuln_label'] = 1  # Add another column filled with 1 (vulnerable code)
    df['threat'] = 'sqli_java'
    df_list.append(df)
    
for file in dirlist_sqli_java:
    df = pd.read_csv("datasets/sqli_java/"+file, names=['code'])
    df['vuln_label'] = 0  # Add another column filled with 1 (vulnerable code)
    df['threat'] = 'sqli_java'
    df_list.append(df)
df = pd.concat(df_list)



print(df.iloc[0])
df.head(10)

NameError: name 'dirlist' is not defined

In [52]:
from sklearn.model_selection import train_test_split

df2 = df

# Slipt data

code = df2['code'].values.astype('str')
y = df2['vuln_label'].values

code_train, code_test, y_train, y_test = train_test_split(
            code, y, test_size=0.25, random_state=1000)


# Vectorize
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(code_train)

X_train = vectorizer.transform(code_train)
X_test  = vectorizer.transform(code_test)
X_train


<1324797x13326 sparse matrix of type '<class 'numpy.int64'>'
	with 3113965 stored elements in Compressed Sparse Row format>

In [53]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)

print("Accuracy:", score)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 0.48420399502716266
